In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
data = pd.read_excel("data.xlsx")

In [ ]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Create a function to build the model
def build_model(hidden_units=100):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_units, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1)  # Output layer with one neuron for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Define the hyperparameters for grid search
param_grid = {
    'epochs': np.arange(25, 101, 25),           # 50,100 ... 500
    'batch_size': [2**i for i in range(3, 7)]  # Powers of 2: 16, 32, 64, 128, 256
}

In [ ]:
# Create the KerasRegressor model
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_model)

In [ ]:
# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=keras_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)

In [ ]:
# Fit the model to the data
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
# Get the best parameters from the grid search
best_params = grid_result.best_params_
print(f'Best Parameters: {best_params}')

In [ ]:
# Make predictions on the testing set using the best model
y_test_pred = grid_result.predict(X_test)
y_train_pred=grid_result.predict(X_train)

In [ ]:
# Calculate Mean Squared Error and R-squared
mse_train=mean_squared_error(y_train,y_train_pred)
r2_train=r2_score(y_train,y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

In [ ]:
print(f'Training MSE: {mse_train:.2f}, R-squared: {r2_train:.2f}')
print(f'Testing MSE: {mse_test:.2f}, R-squared: {r2_test:.2f}')